In [2]:
# Imports
import pandas as pd
from anomaly_tools import HDBAnomalyDetector
from database_tools import PandasDBReader
from fh_config import regression_vars, response_var

In [7]:
# We need to create a PandasDBReader to interface with anomaly detectors.
states = ['FL']
specialty = ['Internal Medicine']
pdb_reader = PandasDBReader("./config.yaml", states, specialty)

In [51]:
# Next, we compute the outlier counts from HDBSCAN.
hdb = HDBAnomalyDetector(regression_vars, response_var, pdb_reader.d_f, use_response_var=True)
min_cluster_size = 10  # Going with 15 for now, it's not too hard to update this later.
hdb.get_outlier_scores(min_size=min_cluster_size)

In [52]:
# We need a DataFrame that contains the actual outlier counts.
counted_df = hdb.get_most_frequent()

In [58]:
print(counted_df.head(20)['last_name'].values)

['QAMAR' 'SEDILLO' 'PAL' 'LEW' 'REDDY' 'CHALASANI' 'LEE' 'KHATIB'
 'LIEBERMAN' 'RUGGIERI' 'RANDALL' 'VANROY' 'ALI' 'VAN DEN BERG' 'MANUBENS'
 'BREDLAU' 'STEIN' 'ATTANTI' 'FERNS' 'GUMMADI']


In [8]:
hdb = HDBAnomalyDetector(regression_vars, response_var, pdb_reader.d_f, use_response_var=True)

In [9]:
top_20_dict = {}
for size in [10, 15, 20, 40, 80, 100, 150]:
    hdb.get_outlier_scores(min_size=size)
    counted_df = hdb.get_most_frequent()
    top_20_dict[size] = list(counted_df.head(20)['last_name'].values)
print(top_20_dict)

/home/dan/.local/lib/python3.5/site-packages/hdbscan/hdbscan_.py:930: RuntimeWarning: invalid value encountered in double_scalars
  self._outlier_scores = outlier_scores(self._condensed_tree)
/home/dan/.local/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


{80: ['RODRIGUEZ', 'COWEN', 'SZYDLOWSKI', 'WASSERMAN', 'HARRIS', 'LECLERCQ', 'MEHLMAN', 'IYER', 'WENK', 'PANCHAL', 'GRASMAN', 'BIRNBAUM', 'LICHTINGER', 'DICKERT', 'COHEN', 'TAWFIK', 'MASI', 'SADHWANI', 'JOSEPH', 'ZUKERBERG'], 20: ['RODRIGUEZ', 'WASSERMAN', 'COWEN', 'HARRIS', 'IYER', 'LECLERCQ', 'WENK', 'MUBARAK', 'GOMER', 'GUPTA', 'COGBURN', 'MALONE', 'KAPADIA', 'SOFI', 'REBELLO', 'BARQUET', 'TAWFIK', 'SZYDLOWSKI', 'MASTERS', 'KHANNA'], 150: ['RODRIGUEZ', 'COWEN', 'WASSERMAN', 'SZYDLOWSKI', 'GOMER', 'IYER', 'HARRIS', 'LECLERCQ', 'GRASMAN', 'GUPTA', 'WENK', 'DICKERT', 'MALONE', 'KHANNA', 'MEHLMAN', 'COGBURN', 'MUBARAK', 'KAPADIA', 'BIRNBAUM', 'LICHTINGER'], 40: ['RODRIGUEZ', 'COWEN', 'WASSERMAN', 'HARRIS', 'IYER', 'LECLERCQ', 'GUPTA', 'MUBARAK', 'KHANNA', 'GOMER', 'MEHLMAN', 'TAWFIK', 'MALONE', 'WENK', 'COGBURN', 'ALBERTI FLOR', 'BARQUET', 'SZYDLOWSKI', 'REBELLO', 'KAPADIA'], 100: ['RODRIGUEZ', 'COWEN', 'SZYDLOWSKI', 'WASSERMAN', 'MEHLMAN', 'IYER', 'HARRIS', 'LECLERCQ', 'WENK', 'PANCHAL

In [7]:
# From this DataFrame we create a sliced DataFrame containing the columns for the new table.
npi_column = list(counted_df.index)
state_column = [geo_info['state'] for geo_info in counted_df['address'].values]
lastname_column = counted_df['last_name']
outlier_count_column = counted_df['outlier_count']
provider_column = [specialty[0] for i in range(len(npi_column))]
for_new_table_data = {'state': state_column, 'lastname': lastname_column, 'provider_type': provider_column, 'outlier_count': outlier_count_column}
for_new_table = pd.DataFrame(data=for_new_table_data, index=npi_column)
for_new_table.to_csv("/home/dan/tx_im.csv")

In [23]:
# Next step: Testing that everything worked
import psycopg2
import yaml

In [25]:
with open("./config.yaml", 'r') as f:
    db_config = yaml.load(f)
con = psycopg2.connect(database=db_config['database_name'],
                       user=db_config['user_name'],
                       password=db_config['password'])
new_query = "SELECT * from provider_anomaly_counts_hdb"
test_df = pd.read_sql_query(new_query, con)
print(test_df.head())

          npi state   lastname provider_type  outlier_count
0  1033145487    FL      QAMAR    Cardiology             48
1  1982680179    FL    SEDILLO    Cardiology             34
2  1376588582    FL        PAL    Cardiology             29
3  1861485757    FL        LEW    Cardiology             21
4  1760477269    FL  CHALASANI    Cardiology             21
